# Lab 6.2: MNIST in CNN

This lab uses the low-level TF API to perform MNIST using CNNs. We will compare the results with our previous model whhich did not use Convolutional Layers

# Part 1: The Low-Level API

Lots of Bookkeeping!  Notice that we are having to keep track of all of the sizes of inputs and outputs, requires you to whip out your calculator

In [ ]:
# Install the package for running tensorboard on google colaboration
!pip install -U tensorboardcolab

In [1]:
import tensorflow as tf
import numpy as np
tf.summary.FileWriterCache.clear()


In [ ]:
# import on colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB == True:
  from tensorboardcolab import *
  tbc=TensorBoardColab()


In [2]:
#Load Mnist data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#Define Variable placeholders

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y_ = tf.placeholder(tf.float32, [None, 10])

# Add summaries to collect information needed for tensorboard
tf.summary.histogram("Weight", W)
tf.summary.histogram("Bias", b)


Instructions for updating:
Colocations handled automatically by placer.


<tf.Tensor 'Bias:0' shape=() dtype=string>

In [4]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


### Picture of the Architecture:

![Illustration of the Model](https://s3.amazonaws.com/elephantscale-public/labs/images/tensorflow-labs/lenet-architecture.png)

In [5]:
# First Convolutional Layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Add summaries to collect information needed for tensorboard
tf.summary.histogram("Weight_conv1", W_conv1)
tf.summary.histogram("Bias_conv1", b_conv1)



<tf.Tensor 'Bias_conv1:0' shape=() dtype=string>

In [6]:
# Second Convolutional Layer

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Add summaries to collect information needed for tensorboard
tf.summary.histogram("Weight_conv2", W_conv2)
tf.summary.histogram("Bias_conv2", b_conv2)


<tf.Tensor 'Bias_conv2:0' shape=() dtype=string>

In [7]:
# Densely Connected Layer

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Add summaries to collect information needed for tensorboard
tf.summary.histogram("Weight_fc1", W_fc1)
tf.summary.histogram("Bias_fc1", b_fc1)


<tf.Tensor 'Bias_fc1:0' shape=() dtype=string>

In [8]:
# Dropout To Reduce Overfitting

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# Readout Layer

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

tf.summary.histogram("Weight_fc2", W_fc2)
tf.summary.histogram("Bias_fc2", b_fc2)
merge=tf.summary.merge_all()


In [10]:
# Train Model
logs_path = '/tmp/tensorflow_logs/example/' #path for writing Tensorboard logs 
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph()) #write the logs 

for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        summary = sess.run(merge )
        summary_writer.add_summary(summary, i )# adding data to logs in every iteration in local path


print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 100, training accuracy 0.1
step 200, training accuracy 0.22
step 300, training accuracy 0.12
step 400, training accuracy 0.16
step 500, training accuracy 0.18
step 600, training accuracy 0.06
step 700, training accuracy 0.14
step 800, training accuracy 0.2
step 900, training accuracy 0.12
step 1000, training accuracy 0.2
step 1100, training accuracy 0.2
step 1200, training accuracy 0.24
step 1300, training accuracy 0.18
step 1400, training accuracy 0.3
step 1500, training accuracy 0.34
step 1600, training accuracy 0.28
step 1700, training accuracy 0.42
step 1800, training accuracy 0.3
step 1900, training accuracy 0.38
test accuracy 0.4729


## Part 2: The tf.keras High Level API



In [4]:
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Standardize to unit norm 0.0 to 1.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

tensorboard = TensorBoard(log_dir='/tmp/tensorflow_logs/example', histogram_freq=1)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5,validation_data=(x_test, y_test), callbacks=[tensorboard])
model.evaluate(x_test, y_test)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 49s 818us/sample - loss: 0.2173 - acc: 0.9339 - val_loss: 0.1036 - val_acc: 0.9684
Epoch 2/5
60000/60000 [==============================] - 47s 787us/sample - loss: 0.0967 - acc: 0.9704 - val_loss: 0.0867 - val_acc: 0.9733
Epoch 3/5
60000/60000 [==============================] - 47s 783us/sample - loss: 0.0690 - acc: 0.9782 - val_loss: 0.0689 - val_acc: 0.9770
Epoch 4/5
60000/60000 [==============================] - 47s 789us/sample - loss: 0.0523 - acc: 0.9832 - val_loss: 0.0615 - val_acc: 0.9809
Epoch 5/5
10000/10000 [==============================] - 1s 139us/sample - loss: 0.0643 - acc: 0.9808


[0.06431571904489537, 0.9808]